# 1. 데이터 정제

## 1-1. 일반/전문의약품 구분

In [ ]:
import pandas as pd
import numpy as np
import re
import time
import requests
from pathlib import Path
from tqdm import tqdm
import json
import PyPDF2
import hashlib
from collections import Counter


In [ ]:
df = pd.read_excel("OpenData_ItemPermitDetail20260122.xls")
print(df)

C:\Users\eddy_\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


                                                  품목명  \
0      중외5%포도당생리식염액(수출명:5%DextroseinnormalsalineInj.)   
1                                          중외5%포도당주사액   
2                                         중외20%포도당주사액   
3                                         중외50%포도당주사액   
4                                       대한포도당주사액(10%)   
...                                               ...   
44040                              카립트현탁액(폴리스티렌설폰산칼슘)   
44041                                       액티브콜드연질캡슐   
44042        텔로셋정80/5/25밀리그램(텔미사르탄, 암로디핀베실산염, 클로르탈리돈)   
44043      텔로셋정80/5/12.5밀리그램(텔미사르탄, 암로디핀베실산염, 클로르탈리돈)   
44044      텔로셋정40/5/12.5밀리그램(텔미사르탄, 암로디핀베실산염, 클로르탈리돈)   

                                                 품목 영문명     품목일련번호 허가/신고구분  \
0           Choongwae 5% Dextrose In Normal Saline Inj.  195500005      신고   
1                              JW 5% Dextrose Injection  195500006      신고   
2                              JW Glucose Injection 20%  195600004      신고   
3  

In [ ]:
df_pham = df[df["전문일반"] == "일반의약품"]

## 1-2. 승인취소된 약품 제거

In [ ]:
df_cleaned = df_pham[["품목명", "전문일반", "용법용량","주성분명", "주의사항", "취소상태", "취소일자", "변경내용"]]
df_cleaned = df_cleaned[df_cleaned["취소상태"] == "정상"]
df_cleaned

,품목명,전문일반,용법용량,주성분명,주의사항,취소상태,취소일자,변경내용
13,활명수,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M279284]L-멘톨|[M050036]육계|[M040548]창출|[M040538...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,"용법·용량, 2022-04-14/사용상주의사항변경(부작용포함), 2019-06-20..."
18,신신티눈고(살리실산반창고)(수출명:SINSINCORNPLASTER),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040264]살리실산,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,"성상변경, 2015-10-30/사용상주의사항변경(부작용포함), 2014-04-25/..."
19,아네모정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M099577]스코폴리아엑스10배산|[M040631]탄산수소나트륨|[M040634...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,"사용상주의사항변경(부작용포함), 2014-11-07/효능효과변경, 2014-11-0..."
20,자모,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040048]길경|[M050058]구아이페네신|[M040482]원지|[M0402...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,"사용상의 주의사항, 2024-09-12/사용상주의사항변경(부작용포함), 2015-0..."
25,페니라민정(클로르페니라민말레산염),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M223211]클로르페니라민말레산염,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,"효능효과변경, 2019-08-26/사용상주의사항변경(부작용포함), 2019-08-2..."
...,...,...,...,...,...,...,...,...
44030,바로온갈근탕액,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040006]감초|[M040071]대추|[M050037]계지|[M040007]건...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,NaN
44032,카푸린플러스비씨정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M223262]티아민질산염|[M040353]아세트아미노펜|[M223155]에페드린...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,NaN
44033,나드정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M222991]레티놀팔미테이트|[M040064]니코틴산아미드|[M255819]토코...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,NaN
44039,무조클린네일라카(테르비나핀염산염),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M258362]테르비나핀염산염,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,정상,NaN,NaN


In [ ]:
df_cleaned = df_cleaned[["품목명", "전문일반", "용법용량","주성분명", "주의사항", "변경내용"]]
df_cleaned

,품목명,전문일반,용법용량,주성분명,주의사항,변경내용
13,활명수,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M279284]L-멘톨|[M050036]육계|[M040548]창출|[M040538...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"용법·용량, 2022-04-14/사용상주의사항변경(부작용포함), 2019-06-20..."
18,신신티눈고(살리실산반창고)(수출명:SINSINCORNPLASTER),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040264]살리실산,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"성상변경, 2015-10-30/사용상주의사항변경(부작용포함), 2014-04-25/..."
19,아네모정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M099577]스코폴리아엑스10배산|[M040631]탄산수소나트륨|[M040634...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"사용상주의사항변경(부작용포함), 2014-11-07/효능효과변경, 2014-11-0..."
20,자모,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040048]길경|[M050058]구아이페네신|[M040482]원지|[M0402...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"사용상의 주의사항, 2024-09-12/사용상주의사항변경(부작용포함), 2015-0..."
25,페니라민정(클로르페니라민말레산염),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M223211]클로르페니라민말레산염,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"효능효과변경, 2019-08-26/사용상주의사항변경(부작용포함), 2019-08-2..."
...,...,...,...,...,...,...
44030,바로온갈근탕액,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040006]감초|[M040071]대추|[M050037]계지|[M040007]건...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,NaN
44032,카푸린플러스비씨정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M223262]티아민질산염|[M040353]아세트아미노펜|[M223155]에페드린...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,NaN
44033,나드정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M222991]레티놀팔미테이트|[M040064]니코틴산아미드|[M255819]토코...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,NaN
44039,무조클린네일라카(테르비나핀염산염),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M258362]테르비나핀염산염,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,NaN


## 1-3. 중복확인

In [ ]:
dup_copy = df_cleaned.duplicated(subset=["품목명"], keep=False)
dups_title = df_cleaned[dup_copy].copy()

print("제거될 중복 행 수:", len(dups_title))
print("중복행: ", dups_title)

제거될 중복 행 수: 0
중복행:  Empty DataFrame
Columns: [품목명, 전문일반, 용법용량, 주성분명, 주의사항, 변경내용]
Index: []


## 1-4. 주의사항, 복용방법 PDF 다운로드

In [ ]:
def sanitize_filename(name: str) -> str:
    
    name = str(name).strip()
    name = re.sub(r'[\\/:*?"<>|]', '_', name)  
    name = name.replace('\n', ' ').replace('\r', ' ')
    name = re.sub(r'\s+', ' ', name).strip()
    return name

def unique_path(path: Path) -> Path:
    
    if not path.exists():
        return path
    stem, suffix = path.stem, path.suffix
    i = 1
    while True:
        p = path.with_name(f"{stem} ({i}){suffix}")
        if not p.exists():
            return p
        i += 1

def download_pdf(session: requests.Session, url: str, save_path: Path, headers: dict, timeout=40):
    r = session.get(url, headers=headers, timeout=timeout, stream=True)
    r.raise_for_status()

    
    it = r.iter_content(chunk_size=16)
    first = next(it)
    if not first.startswith(b"%PDF"):
        raise ValueError("Response is not a PDF (no %PDF header).")

    with open(save_path, "wb") as f:
        f.write(first)
        for chunk in r.iter_content(chunk_size=1024 * 256):
            if chunk:
                f.write(chunk)

def batch_download(df: pd.DataFrame,
                   name_col="품목명",
                   dose_url_col="용법용량",
                   warn_url_col="주의사항",
                   out_dir="pdfs",
                   sleep_sec=0.2,
                   timeout=40):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/pdf,*/*",
        "Referer": "https://nedrug.mfds.go.kr/",
    }

    ok, fail = 0, 0
    failed_names = [] 

    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="품목 처리", unit="drug"):
        product = sanitize_filename(row[name_col])

        for section, url_col in [("용법용량", dose_url_col), ("주의사항", warn_url_col)]:
            url = row.get(url_col, "")
            url = "" if pd.isna(url) else str(url).strip()
            if not url or url.lower() == "nan":
                continue

            base_name = f"{product}__{section}.pdf"
            save_path = unique_path(out_dir / base_name)

            try:
                download_pdf(session, url, save_path, headers=headers, timeout=timeout)
                ok += 1
            except Exception as e:
                fail += 1
                failed_names.append(str(row[name_col]))  
                print(f"[FAIL] row={idx} | {row[name_col]} ({section}) | {e}")

            time.sleep(sleep_sec)

    
    failed_unique = sorted(set([x.strip() for x in failed_names if str(x).strip()]))

    print(f"\nDone. OK={ok}, FAIL={fail}, dir={out_dir.resolve()}")
    print(f"실패한 품목명(중복 제거) 수: {len(failed_unique)}")
    if failed_unique:
        print("예시(최대 30개):")
        for name in failed_unique[:30]:
            print(" -", name)

    return failed_unique


In [ ]:
failed_unique = batch_download(df_cleaned, out_dir=r"C:\Users\eddy_\Documents\LLM\pdfs", sleep_sec=0.2)

품목 처리:  44%|████▍     | 3842/8650 [43:56<56:34,  1.42drug/s]  

[FAIL] row=17441 | 퓨어에이드나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명:Pure Aid Nasal Spray Original,  Pure Aid Nasal Spray Extra Moisturizing, ASSURED NASAL RELIEF SPRAY PUMP MIST ANTI-DRIP ORIGINAL Oxymetazoline HCI-Nasal Decongestant,

VM PLUS Nasal Spray, Family Care Nasal Spray Original,

Family Care Severe Congestion Nasal Spray ,

Leader Extra Moisturizing Nasal Spray Original,

Leader Severe Congestion Nasal Spray) (용법용량) | [Errno 22] Invalid argument: 'C:\\Users\\eddy_\\Documents\\LLM\\pdfs\\퓨어에이드나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명_Pure Aid Nasal Spray Original, Pure Aid Nasal Spray Extra Moisturizing, ASSURED NASAL RELIEF SPRAY PUMP MIST ANTI-DRIP ORIGINAL Oxymetazoline HCI-Nasal Decongestant, VM PLUS Nasal Spray, Family Care Nasal Spray Original, Family Care Severe Congestion Nasal Spray , Leader Extra Moisturizing Nasal Spray Original, Leader Severe Congestion Nasal Spray)__용법용량.pdf'
[FAIL] row=17441 | 퓨어에이드나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명:Pure Aid Nasal Spray Original,  Pure Aid Nasal Spray Extr

품목 처리: 100%|██████████| 8650/8650 [2:05:13<00:00,  1.15drug/s]  


Done. OK=17298, FAIL=2, dir=C:\Users\eddy_\Documents\LLM\pdfs
실패한 품목명(중복 제거) 수: 1
예시(최대 30개):
 - 퓨어에이드나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명:Pure Aid Nasal Spray Original,  Pure Aid Nasal Spray Extra Moisturizing, ASSURED NASAL RELIEF SPRAY PUMP MIST ANTI-DRIP ORIGINAL Oxymetazoline HCI-Nasal Decongestant,

VM PLUS Nasal Spray, Family Care Nasal Spray Original,

Family Care Severe Congestion Nasal Spray ,

Leader Extra Moisturizing Nasal Spray Original,

Leader Severe Congestion Nasal Spray)


## 1-5. 다운로드 실패 파일 제거

In [ ]:
df_cleaned.loc[17438:17443]

,품목명,전문일반,용법용량,주성분명,주의사항,변경내용
17441,퓨어에이드나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명:Pure Aid N...,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M223162]옥시메타졸린염산염,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"제품명칭변경, 2013-05-31/성상변경, 2013-05-31/제품명칭변경, 20..."


In [ ]:
df_cleaned = df_cleaned.drop(index=17441)
df_cleaned.loc[17441]

KeyError: 17441

In [ ]:
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned

,품목명,전문일반,용법용량,주성분명,주의사항,변경내용
0,활명수,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M279284]L-멘톨|[M050036]육계|[M040548]창출|[M040538...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"용법·용량, 2022-04-14/사용상주의사항변경(부작용포함), 2019-06-20..."
1,신신티눈고(살리실산반창고)(수출명:SINSINCORNPLASTER),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040264]살리실산,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"성상변경, 2015-10-30/사용상주의사항변경(부작용포함), 2014-04-25/..."
2,아네모정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M099577]스코폴리아엑스10배산|[M040631]탄산수소나트륨|[M040634...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"사용상주의사항변경(부작용포함), 2014-11-07/효능효과변경, 2014-11-0..."
3,자모,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040048]길경|[M050058]구아이페네신|[M040482]원지|[M0402...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"사용상의 주의사항, 2024-09-12/사용상주의사항변경(부작용포함), 2015-0..."
4,페니라민정(클로르페니라민말레산염),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M223211]클로르페니라민말레산염,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,"효능효과변경, 2019-08-26/사용상주의사항변경(부작용포함), 2019-08-2..."
...,...,...,...,...,...,...
8644,바로온갈근탕액,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M040006]감초|[M040071]대추|[M050037]계지|[M040007]건...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,NaN
8645,카푸린플러스비씨정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M223262]티아민질산염|[M040353]아세트아미노펜|[M223155]에페드린...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,NaN
8646,나드정,일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M222991]레티놀팔미테이트|[M040064]니코틴산아미드|[M255819]토코...,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,NaN
8647,무조클린네일라카(테르비나핀염산염),일반의약품,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,[M258362]테르비나핀염산염,https://nedrug.mfds.go.kr/pbp/cmn/pdfDownload/...,NaN


## 1-6. 메타데이터 생성

In [ ]:
df_meta = df_cleaned[['품목명','전문일반','주성분명']]
df_meta

,품목명,전문일반,주성분명
0,활명수,일반의약품,[M279284]L-멘톨|[M050036]육계|[M040548]창출|[M040538...
1,신신티눈고(살리실산반창고)(수출명:SINSINCORNPLASTER),일반의약품,[M040264]살리실산
2,아네모정,일반의약품,[M099577]스코폴리아엑스10배산|[M040631]탄산수소나트륨|[M040634...
3,자모,일반의약품,[M040048]길경|[M050058]구아이페네신|[M040482]원지|[M0402...
4,페니라민정(클로르페니라민말레산염),일반의약품,[M223211]클로르페니라민말레산염
...,...,...,...
8644,바로온갈근탕액,일반의약품,[M040006]감초|[M040071]대추|[M050037]계지|[M040007]건...
8645,카푸린플러스비씨정,일반의약품,[M223262]티아민질산염|[M040353]아세트아미노펜|[M223155]에페드린...
8646,나드정,일반의약품,[M222991]레티놀팔미테이트|[M040064]니코틴산아미드|[M255819]토코...
8647,무조클린네일라카(테르비나핀염산염),일반의약품,[M258362]테르비나핀염산염


# 2. 데이터 스캔 및 청킹

## 2-1. PDF 스캔 및 청킹

In [ ]:
def extract_text_from_pdf(pdf_path):
    pages_text = []
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text() or ""
            text = text.strip()
            if text and len(text) > 10:
                pages_text.append(text)
    return "\n\n".join(pages_text)

def normalize_text(t: str) -> str:
    t = t.replace("\r", "\n")
    t = re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", " ", t)  
    t = re.sub(r"[ \t]+", " ", t)
    t = re.sub(r"\n{3,}", "\n\n", t)
    return t.strip()

def parse_filename(path: Path):
    stem = path.stem
    if "__" not in stem:
        return None, None

    drug, section = stem.split("__", 1)

    if "용법" in section:
        section = "용법용량"
    elif "주의" in section:
        section = "주의사항"
    else:
        return None, None

    return drug, section

def build_two_chunks(pdf_dir, out_jsonl="two_chunks.jsonl"):
    pdf_dir = Path(pdf_dir)
    pdf_files = sorted(pdf_dir.glob("*.pdf"))

    drug_map = {}
    for p in tqdm(pdf_files, desc="PDF 스캔(파일명 파싱)"):
        drug, section = parse_filename(p)
        if not drug or not section:
            continue
        drug_map.setdefault(drug, {}).setdefault(section, []).append(p)

    total_drugs = len(drug_map)

    ok_records = 0
    empty_drugs = []  

    with open(out_jsonl, "w", encoding="utf-8") as f:
        for drug, sections in tqdm(drug_map.items(), total=total_drugs, desc="약물별 텍스트 추출/저장"):
            dose_texts = []
            for p in sections.get("용법용량", []):
                t = normalize_text(extract_text_from_pdf(p))
                if t:
                    dose_texts.append(t)
            dose_text = "\n\n".join(dose_texts).strip()

            warn_texts = []
            for p in sections.get("주의사항", []):
                t = normalize_text(extract_text_from_pdf(p))
                if t:
                    warn_texts.append(t)
            warn_text = "\n\n".join(warn_texts).strip()

            wrote_any = False

            if dose_text:
                rec = {
                    "drug_name": drug,
                    "section": "용법용량",
                    "text": f"【약물명】{drug}\n【섹션】용법용량\n---\n{dose_text}"
                }
                f.write(json.dumps(rec, ensure_ascii=False) + "\n")
                ok_records += 1
                wrote_any = True

            if warn_text:
                rec = {
                    "drug_name": drug,
                    "section": "주의사항",
                    "text": f"【약물명】{drug}\n【섹션】주의사항\n---\n{warn_text}"
                }
                f.write(json.dumps(rec, ensure_ascii=False) + "\n")
                ok_records += 1
                wrote_any = True

            if not wrote_any:
                empty_drugs.append(drug)  

    print(f"Saved to: {Path(out_jsonl).resolve()}")
    print(f"생성된 레코드 수(줄 수): {ok_records}")
    print(f"텍스트가 비어 스킵된 약물 수: {len(empty_drugs)}")

    
    if empty_drugs:
        print("\n--- 스킵된 약물명 일부(최대 50개) ---")
        for name in empty_drugs[:50]:
            print(name)
        if len(empty_drugs) > 50:
            print(f"... (총 {len(empty_drugs)}개 중 50개만 표시)")

    return empty_drugs  

build_two_chunks(pdf_dir=r"C:\Users\eddy_\Documents\LLM\pdfs", out_jsonl="two_chunks.jsonl")


약물별 텍스트 추출/저장: 100%|██████████| 8649/8649 [05:39<00:00, 25.49it/s]

Saved to: C:\Users\eddy_\Documents\LLM\RAG portfolio\two_chunks.jsonl
생성된 레코드 수(줄 수): 17298
텍스트가 비어 스킵된 약물 수: 0


[]

In [ ]:
records = []
with open("two_chunks.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

print("10번째 레코드:", records[11])         
print("10번째 텍스트:", records[11]["text"]) 

10번째 레코드: {'drug_name': '5-엠씨크림(리도카인)', 'section': '주의사항', 'text': '【약물명】5-엠씨크림(리도카인)\n【섹션】주의사항\n---\n사용상의주의사항\n1. 다음 환자에는 투여하지 말 것\n1) 본인 또는 상대방이 이 약 또는 기타의 아미드계 국소마취제에 과민증인 경우\n2) 적용 부위에 외상 또는 염증 환자\n3) 상대방이 임신중인 경우\n2. 다음 환자에는 신중히 투여할 것\n1) 간·신질환 환자\n3. 일반적 주의\n1) 본인 또는 상대방에 발진 또는 자극이 나타나는 경우에는 이 약의 사용을 중지하고 증상이 계속되는 경우\n에는 의사와 상담한다.\n2) 의사의 감독을 받지 않고 3개월 이상 이 약을 사용하지 않는다.\n4. 적용상의 주의\n1) 외용으로만 사용하며 눈 또는 코에 사용하지 않는다.\n2) 동일한 부위에 연속해서 도포하지 않는 것이 바람직하다.\n5. 보관 및 취급상의 주의사항\n1) 유·소아의 손이 닿지 않는 곳에 보관한다.\n2) 40℃ 이상의 고온 또는 저온의 장소, 일광이 닿는 곳에 보관하지 않는다.\n3) 사용 후 불속에 버리지 않는다.'}
10번째 텍스트: 【약물명】5-엠씨크림(리도카인)
【섹션】주의사항
---
사용상의주의사항
1. 다음 환자에는 투여하지 말 것
1) 본인 또는 상대방이 이 약 또는 기타의 아미드계 국소마취제에 과민증인 경우
2) 적용 부위에 외상 또는 염증 환자
3) 상대방이 임신중인 경우
2. 다음 환자에는 신중히 투여할 것
1) 간·신질환 환자
3. 일반적 주의
1) 본인 또는 상대방에 발진 또는 자극이 나타나는 경우에는 이 약의 사용을 중지하고 증상이 계속되는 경우
에는 의사와 상담한다.
2) 의사의 감독을 받지 않고 3개월 이상 이 약을 사용하지 않는다.
4. 적용상의 주의
1) 외용으로만 사용하며 눈 또는 코에 사용하지 않는다.
2) 동일한 부위에 연속해서 도포하지 않는 것이 바람직하다.
5. 보관 및 취급상의 주의사항
1) 유·소아의 손이 닿지 않는 곳에 

- 주의사항 섹션이 너무 길어 한번 더 청킹

In [ ]:
def split_by_len(text: str, max_chars: int = 1500, overlap: int = 120):
    text = str(text).strip()
    if not text:
        return []
    if len(text) <= max_chars:
        return [text]

    step = max_chars - overlap
    chunks = []
    for i in range(0, len(text), step):
        chunks.append(text[i:i+max_chars])
    return chunks


def sanitize_filename(name: str) -> str:
    
    name = str(name).strip()

    name = re.sub(r'[\\/:*?"<>|]', '_', name)

    name = name.replace('\n', ' ').replace('\r', ' ')
    name = re.sub(r'\s+', ' ', name).strip()

    name = re.sub(r'_+', '_', name)    
    name = name.strip('_')   

    return name



def build_meta_map(df_meta):
   
    meta = {}
    collisions = {}
    for _, row in df_meta.iterrows():
        raw_name = str(row["품목명"]).strip()
        key = sanitize_filename(raw_name)
        if not key:
            continue

        item = {
            "pro_type": str(row["전문일반"]).strip(),
            "active_ingredient": str(row["주성분명"]).strip(),
            "raw_name": raw_name,
        }

        if key in meta:
              collisions.setdefault(key, []).append(raw_name)
              continue
        meta[key] = item

    return meta, collisions


def strip_header(text: str) -> str:

    if "---\n" in text:
        return text.split("---\n", 1)[1].strip()
    if "---\r\n" in text:
        return text.split("---\r\n", 1)[1].strip()
    return text.strip()


def _split_by_paragraph_len(text: str, max_chars: int):
    text = str(text).strip()
    if not text:
        return []

 
    paras = [p.strip() for p in text.split("\n\n") if p.strip()]

    if len(paras) <= 1:
        paras = [p.strip() for p in text.split("\n") if p.strip()]

    chunks = []
    cur = ""
    for p in paras:
        
        candidate = p if not cur else (cur + "\n" + p)

        if len(candidate) <= max_chars:
            cur = candidate
            continue

        if cur:
            chunks.append(cur)
            cur = ""

        if len(p) > max_chars:
            for i in range(0, len(p), max_chars):
                chunks.append(p[i:i+max_chars])
        else:
            cur = p

    if cur:
        chunks.append(cur)

    return chunks


def split_warn_by_major_groups(text: str, group_size: int = 1, max_chars: int = 1500):
    
    text = text.strip()
    if not text:
        return []

    major_re = re.compile(r"(?m)^\s*\d{1,2}\.\s+")  
    matches = list(major_re.finditer(text))

    if len(matches) < 2:
        return [text] if len(text) <= max_chars else _split_by_paragraph_len(text, max_chars)

    
    starts = [m.start() for m in matches] + [len(text)]
    sections = []
    for s, e in zip(starts, starts[1:]):
        sec = text[s:e].strip()
        if sec:
            sections.append(sec)

   
    grouped = []
    for i in range(0, len(sections), group_size):
        grouped.append("\n\n".join(sections[i:i+group_size]).strip())

    final = []
    for g in grouped:
        if len(g) <= max_chars:
            final.append(g)
        else:
            final.extend(_split_by_paragraph_len(g, max_chars))

    return final


def upgrade_two_chunks_major(
    two_chunks_path: str,
    df_meta,
    out_path: str = "real_final_chunks.jsonl",
    group_size: int = 1,
    max_warn_chars: int = 1500
):
    meta_map, collisions = build_meta_map(df_meta)  

    with open(two_chunks_path, "r", encoding="utf-8") as f:
        total_lines = sum(1 for _ in f)

    out_cnt = 0
    missing_meta = set()

    with open(two_chunks_path, "r", encoding="utf-8") as fin, open(out_path, "w", encoding="utf-8") as fout:
        for line in tqdm(fin, total=total_lines, desc="two_chunks 후처리"):
            obj = json.loads(line)

            drug = str(obj.get("drug_name", "")).strip()
            section = str(obj.get("section", "")).strip()
            text = obj.get("text", "")

            if not drug or not section or not text:
                continue

            drug_key = sanitize_filename(drug)

            base_meta = meta_map.get(drug_key)
            if not base_meta:
                missing_meta.add(drug)
                base_meta = {"pro_type": "", "active_ingredient": "", "raw_name": drug}

            display_name = base_meta.get("raw_name", drug)

            drug_id = hashlib.sha1(drug_key.encode("utf-8")).hexdigest()[:12]

            body = strip_header(text)

            if section == "주의사항":
                parts = split_warn_by_major_groups(
                    body, group_size=group_size, max_chars=max_warn_chars
                )
                if not parts:
                    continue

                for i, part in enumerate(parts, 1):
                    chunk_id = f"{drug_id}-warn-{i}"
                    rec = {
                        "drug_id": drug_id,
                        "chunk_id": chunk_id,

                        "drug_name": display_name,      
                        "drug_name_key": drug_key,      # sanitize된 키
                        "pro_type": base_meta["pro_type"],
                        "active_ingredient": base_meta["active_ingredient"],

                        "section": "주의사항",
                        "sub_index": i,
                        "sub_count": len(parts),

                        "text": (
                            f"【약물명】{display_name}\n"
                            f"【주성분】{base_meta['active_ingredient']}\n"
                            f"【구분】{base_meta['pro_type']}\n"
                            f"【섹션】주의사항 ({i}/{len(parts)})\n"
                            f"---\n{part}"
                        )
                    }
                    fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    out_cnt += 1

            elif section == "용법용량":
                parts = split_by_len(body, max_chars=max_warn_chars, overlap=120)
                if not parts:
                    continue

                for i, part in enumerate(parts, 1):
                    chunk_id = f"{drug_id}-dose-{i}"
                    rec = {
                        "drug_id": drug_id,
                        "chunk_id": chunk_id,

                        "drug_name": display_name,
                        "drug_name_key": drug_key,
                        "pro_type": base_meta["pro_type"],
                        "active_ingredient": base_meta["active_ingredient"],

                        "section": "용법용량",
                        "sub_index": i,
                        "sub_count": len(parts),

                        "text": (
                            f"【약물명】{display_name}\n"
                            f"【주성분】{base_meta['active_ingredient']}\n"
                            f"【구분】{base_meta['pro_type']}\n"
                            f"【섹션】용법용량 ({i}/{len(parts)})\n"
                            f"---\n{part}"
                        )
                    }
                    fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    out_cnt += 1

            else:
                continue

    print(f"\nSaved: {out_path}")
    print(f"총 생성 레코드 수: {out_cnt}")

    if collisions:
        print(f"sanitize 충돌 key 수: {len(collisions)}")

    if missing_meta:
        missing_list = sorted(missing_meta)
        print(f"CSV(df_meta)에서 메타 못 찾은 drug_name 수: {len(missing_list)}")
        print(f"CSV(df_meta)에서 메타 못 찾은 drug_name: {missing_list}")
        print("예시(최대 30개):")
        for name in missing_list[:30]:
            print(" -", name)


meta_map, collisions = build_meta_map(df_meta)

missing = 0
total = 0
with open("two_chunks.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        drug = obj["drug_name"]
        total += 1
        if sanitize_filename(drug) not in meta_map:
            missing += 1

print("총 라인:", total)
print("메타 매칭 실패 라인:", missing)
print("sanitize key 충돌 수:", len(collisions))


upgrade_two_chunks_major(
    two_chunks_path="two_chunks.jsonl",
    df_meta = df_meta,
    out_path="real_final_chunks.jsonl",
    group_size=1,
    max_warn_chars=1500
)


총 라인: 17298
메타 매칭 실패 라인: 0
sanitize key 충돌 수: 0


two_chunks 후처리: 100%|██████████| 17298/17298 [00:01<00:00, 9469.69it/s] 


Saved: real_final_chunks.jsonl
총 생성 레코드 수: 64015


청크 문자열 길이 확인

In [ ]:
df_chunk = pd.read_json("real_final_chunks.jsonl", lines=True, encoding="utf-8")

lens = df_chunk["text"].astype(str).map(len).to_numpy()

print("청크 개수:", len(lens))
print("평균 문자 수:", float(np.mean(lens)))
print("중앙값:", float(np.median(lens)))
print("p90:", float(np.percentile(lens, 90)))
print("p95:", float(np.percentile(lens, 95)))
print("p99:", float(np.percentile(lens, 99)))
print("최대:", int(np.max(lens)))

청크 개수: 64015
평균 문자 수: 399.3460751386394
중앙값: 287.0
p90: 838.0
p95: 1105.2999999999956
p99: 1533.0
최대: 1887


In [ ]:
df_chunk["char_len"] = df_chunk["text"].astype(str).map(len)

df_chunk.sort_values("char_len", ascending=False)[["section","drug_name","chunk_id","char_len"]].head(20)

,section,drug_name,chunk_id,char_len
5125,주의사항,뉴먼트에이스연질캡슐,167d72d332ad-warn-3,1887
22860,주의사항,삼타민에프연질캡슐(수출용),0cfec9c1508a-warn-1,1831
44916,주의사항,진셀몬큐텐정,d18209495726-warn-3,1828
21225,주의사항,비맥스제트정,b3499f626e9c-warn-2,1809
29369,주의사항,아로나민씨플러스정,bebabf523a22-warn-2,1794
44898,주의사항,진생민연질캡슐(수출용),f2a74ec23b35-warn-3,1792
21107,주의사항,비나폴로100정,baa63126cffd-warn-2,1780
15232,주의사항,메가원플러스정,1237b3cbc626-warn-2,1763
21204,주의사항,비맥스비비정,43c8ea90eea6-warn-2,1759
55068,주의사항,파워액티브정,b14673157cf2-warn-2,1758


헤더 제외한 나머지 부분 확인

In [ ]:
def body_only(s: str) -> str:
    s = str(s)
    if "---\n" in s:
        return s.split("---\n", 1)[1]
    if "---\r\n" in s:
        return s.split("---\r\n", 1)[1]
    return s

df["body_len"] = df["text"].astype(str).map(lambda x: len(body_only(x)))
warn = df[df["section"]=="주의사항"]

print("주의사항 body_len max:", int(warn["body_len"].max()))
print(warn.sort_values("body_len", ascending=False)[["drug_name","chunk_id","body_len"]].head(20))


주의사항 body_len max: 1500
                    drug_name              chunk_id  body_len
7756        덱시푸루펜연질캡슐(덱시부프로펜)  329851beb278-warn-15      1500
7754        덱시푸루펜연질캡슐(덱시부프로펜)  329851beb278-warn-13      1500
14809        맥시쿨펜연질캡슐(덱시부프로펜)  740a81ea5fa3-warn-15      1500
14807        맥시쿨펜연질캡슐(덱시부프로펜)  740a81ea5fa3-warn-13      1500
37819        옵티제로연질캡슐(덱시부프로펜)  5b3a05707e4a-warn-15      1500
7734            덱시파인정(덱시부프로펜)  aeed930203bd-warn-13      1500
14821           맥시펜시럽(덱시부프로펜)   344d6136f8f5-warn-6      1500
7709            덱시탑시럽(덱시부프로펜)   0d585feba945-warn-7      1500
7708            덱시탑시럽(덱시부프로펜)   0d585feba945-warn-6      1500
7822              덱펜정(덱시부프로펜)  5b5bc0ea57c0-warn-15      1500
14771  맥시부펜이알정300밀리그램(덱시부프로펜)  1284d5ec6980-warn-13      1500
7820              덱펜정(덱시부프로펜)  5b5bc0ea57c0-warn-13      1500
14841            맥쎈연질캡슐(나프록센)   54279801517a-warn-7      1500
14822           맥시펜시럽(덱시부프로펜)   344d6136f8f5-warn-7      1500
7696          덱시원연질캡슐(덱시부프로펜)  82576f44ed39-wa

## 2-2 품질확인

In [ ]:
required = ["drug_id","chunk_id","drug_name","drug_name_key","section","text"]
missing = {}
for c in required:
    missing[c] = (df_chunk[c].isna() | (df_chunk[c].astype(str).str.strip()=="")).mean()
print("Missing rate:", missing)

Missing rate: {'drug_id': np.float64(0.0), 'chunk_id': np.float64(0.0), 'drug_name': np.float64(0.0), 'drug_name_key': np.float64(0.0), 'section': np.float64(0.0), 'text': np.float64(0.0)}


2. chunk_id 유일성 검증

In [ ]:
dup_chunk_id = df_chunk["chunk_id"].duplicated().sum()
print("Duplicated chunk_id:", dup_chunk_id)

Duplicated chunk_id: 0


3. 그룹 내부에서 sub_count 혼재 여부, row 수 일치 여부

In [ ]:
g = df_chunk.groupby(["drug_id", "section"], dropna=False)
subcount_unique = g["sub_count"].nunique()
bad_subcount_unique = subcount_unique[subcount_unique > 1]

rows = g.size()
subcount_first = g["sub_count"].first()
bad_subcount_mismatch = (rows != subcount_first)

print("Groups with varying sub_count:", len(bad_subcount_unique))
print("Groups where rows != sub_count:", int(bad_subcount_mismatch.sum()))

Groups with varying sub_count: 0
Groups where rows != sub_count: 0


4. sub_index 정합성 검증

In [ ]:
def check_sub_index_1start(gr):
    idx = gr["sub_index"].dropna().astype(int).tolist()
    if len(idx) == 0:
        return True 

    sc = int(gr["sub_count"].iloc[0])

    if any(i < 1 or i > sc for i in idx):
        return True

    if len(idx) != len(set(idx)):
        return True

    s = set(idx)
    if any(i not in s for i in range(1, sc + 1)):
        return True

    return False 

bad_index_groups = g.apply(check_sub_index_1start)

print("Groups with sub_index issues (oor/dup/missing):", int(bad_index_groups.sum()))

Groups with sub_index issues (oor/dup/missing): 0


C:\Users\eddy_\AppData\Local\Temp\ipykernel_32548\2556409579.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bad_index_groups = g.apply(check_sub_index_1start)


5. 길이분포

In [ ]:
df_chunk["len_chars"] = df_chunk["text"].astype(str).str.len()
print(df_chunk["len_chars"].describe(percentiles=[.9,.95,.99]))

count    64015.000000
mean       399.346075
std        311.673475
min         65.000000
50%        287.000000
90%        838.000000
95%       1105.300000
99%       1533.000000
max       1887.000000
Name: len_chars, dtype: float64


6. 완전중복 확인

In [ ]:
dup_text_rate = df_chunk["text"].duplicated().mean()
print("Exact duplicate text rate:", dup_text_rate)

Exact duplicate text rate: 0.0


7. 약물내부 완전중복

In [ ]:
dup_within = df_chunk.duplicated(subset=["drug_id","section","text"]).mean()
print("Duplicate within drug_id/section:", dup_within)

Duplicate within drug_id/section: 0.0


# 3. 샘플 추출

## 3-1. context 랜덤선택

- 용법용량보다 주의사항에 대한 문의가 많을것으로 예상해, 용법용량만 500개의 약물로 context 구성

In [ ]:
def split_body(text: str) -> str:
    if not isinstance(text, str):
        return ""
    if "---" in text:
        return text.split("---", 1)[1].strip()
    return text.strip()


def select_dosage_drugs(df_chunk: pd.DataFrame, n: int = 500, seed: int = 42) -> set:
    dosage_drugs = (
        df_chunk.loc[df_chunk["section"] == "용법용량", "drug_id"]
        .dropna()
        .drop_duplicates()
        .tolist()
    )
    if len(dosage_drugs) < n:
        raise ValueError(f"'용법용량'이 있는 약물이 {len(dosage_drugs)}개뿐이라 {n}개를 뽑을 수 없음.")

    return set(pd.Series(dosage_drugs).sample(n=n, random_state=seed).tolist())


def stable_seed(drug_id: str, base_seed: int = 42) -> int:
    h = hashlib.md5(drug_id.encode("utf-8")).hexdigest()
    return base_seed + (int(h[:8], 16) % 1_000_000)

def pick_warnings_balanced(warn_df: pd.DataFrame, warn_k: int, rs: int) -> pd.DataFrame:
 
    if warn_df is None or len(warn_df) == 0 or warn_k <= 0:
        return warn_df.iloc[0:0]

    warn_df = warn_df.sort_values(["sub_index", "chunk_id"]).reset_index(drop=True)

    n = len(warn_df)
    if n <= warn_k:
        return warn_df 

    min_si = int(warn_df["sub_index"].min())
    max_si = int(warn_df["sub_index"].max())

    if min_si == max_si or pd.isna(min_si) or pd.isna(max_si):
        bounds = [int(round(i * n / warn_k)) for i in range(warn_k + 1)]
        picks = []
        for i in range(warn_k):
            s, e = bounds[i], bounds[i + 1]
            if s >= e:
                continue
            seg = warn_df.iloc[s:e]
            picks.append(seg.sample(n=1, random_state=rs + i))
        picked = pd.concat(picks).drop_duplicates(subset=["chunk_id"])
        return picked.sort_values(["sub_index", "chunk_id"])

    span = (max_si - min_si + 1)
    step = span / warn_k

    picks = []
    for i in range(warn_k):
        lo = min_si + int(round(i * step))
        hi = min_si + int(round((i + 1) * step))
        if i == warn_k - 1:
            hi = max_si + 1  

        seg = warn_df[(warn_df["sub_index"] >= lo) & (warn_df["sub_index"] < hi)]
        if len(seg) == 0:
            continue
        picks.append(seg.sample(n=1, random_state=rs + i))

    picked = pd.concat(picks) if picks else warn_df.iloc[0:0]
    picked = picked.drop_duplicates(subset=["chunk_id"])

    need = warn_k - len(picked)
    if need > 0:
        remaining = warn_df[~warn_df["chunk_id"].isin(picked["chunk_id"].tolist())]
        if len(remaining) > 0:
            extra = remaining.sample(n=min(need, len(remaining)), random_state=rs + 999)
            picked = pd.concat([picked, extra]).drop_duplicates(subset=["chunk_id"])

    return picked.sort_values(["sub_index", "chunk_id"])


def build_context_pack(
    g: pd.DataFrame,
    mode: str,
    warn_k: int = 1,
    base_seed: int = 42,
    dose_k: int = 1
) -> dict:
    parts = []
    source_chunk_ids = []

    drug_name = str(g["drug_name"].iloc[0]).strip()
    parts.append(f"{drug_name} :")

    if mode == "dosage":
        dose = g[g["section"] == "용법용량"].sort_values(["sub_index", "chunk_id"])
        if len(dose) > 0:
            picked = dose.head(min(dose_k, len(dose)))
            for i in range(len(picked)):
                body = split_body(picked.iloc[i]["text"])
                if body:
                    parts.append(f"### 용법용량({i+1})\n" + body)
                    source_chunk_ids.append(picked.iloc[i]["chunk_id"])
        qa_type_hint = "dosage"

    elif mode == "warning":
        warn = g[g["section"] == "주의사항"].sort_values(["sub_index", "chunk_id"])
        if len(warn) > 0:
            rs = stable_seed(str(g["drug_id"].iloc[0]), base_seed)

            picked = pick_warnings_balanced(warn, warn_k=warn_k, rs=rs)

            for i in range(len(picked)):
                body = split_body(picked.iloc[i]["text"])
                if body:
                    parts.append(f"### 주의사항({i+1})\n" + body)
                    source_chunk_ids.append(picked.iloc[i]["chunk_id"])

        qa_type_hint = "warning"

    else:
        raise ValueError("mode는 'dosage' 또는 'warning'만 가능")

    context = "\n\n".join(parts).strip()

    return {
        "drug_id": g["drug_id"].iloc[0],
        "drug_name_key": g["drug_name_key"].iloc[0],
        "drug_name": g["drug_name"].iloc[0],
        "active_ingredient": g["active_ingredient"].iloc[0],
        "pro_type": g["pro_type"].iloc[0],
        "context_mode": mode,
        "qa_type_hint": qa_type_hint,
        "source_chunk_ids": source_chunk_ids,
        "context": context,
    }


def build_mixed_context_packs(df_chunk: pd.DataFrame,
                             dosage_n: int = 500,
                             base_seed: int = 42,
                             warn_k: int = 1) -> tuple[pd.DataFrame, set]:
    dosage_drug_ids = select_dosage_drugs(df_chunk, n=dosage_n, seed=base_seed)

    packs = []
    for drug_id, g in df_chunk.groupby("drug_id", sort=False):
        mode = "dosage" if drug_id in dosage_drug_ids else "warning"
        pack = build_context_pack(
            g,
            mode=mode,
            warn_k=warn_k,
            base_seed=base_seed
        )
        if pack["context"]:
            packs.append(pack)

    df_pack = pd.DataFrame(packs).reset_index(drop=True)
    return df_pack, dosage_drug_ids


df_pack, dosage_drug_ids = build_mixed_context_packs(
    df_chunk,
    dosage_n=500,  
    base_seed=42,
    warn_k=1  
)

print("총 context 생성 약물 수:", df_pack["drug_id"].nunique())
print("용법용량 context 약물 수:", (df_pack["context_mode"] == "dosage").sum())
print("주의사항 context 약물 수:", (df_pack["context_mode"] == "warning").sum())

print("\n[dosage 예시]\n", df_pack[df_pack["context_mode"]=="dosage"].iloc[0]["context"][:400])
print("\n[warning 예시]\n", df_pack[df_pack["context_mode"]=="warning"].iloc[0]["context"][:400])

pd.Series(sorted(dosage_drug_ids)).to_csv("dosage_drug_ids_500.csv", index=False, header=["drug_id"])
df_pack.to_json("context_packs.jsonl", orient="records", lines=True, force_ascii=False)
print("saved -> final_dosage_drug_ids_500.csv, final_context_packs.jsonl")


총 context 생성 약물 수: 8649
용법용량 context 약물 수: 500
주의사항 context 약물 수: 8149

[dosage 예시]
 가네리버연질캡슐350mg(밀크시슬열매건조엑스) :

### 용법용량(1)
용법용량
성인 1일 1회, 1회 1캡슐 복용

[warning 예시]
 1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈요오드)(청포도향) :

### 주의사항(1)
3. 다음과 같은 경우 이 약의 사용을 즉각 중지하고 의사, 치과의사, 약사와 상의할 것. 상담 시 가능한 한 이 첨
부문서를 소지할 것.
1) 과민증: 과민증(발진 등), 아나필락시양 증상이 나타나는 경우에는 사용을 중단한다.
2) 구강(입안) : 때때로 구강(입안), 인두의 자극감, 드물게 구강(입안)점막미란(짓무름), 입안의 헐음 등의 증
상이 나타날 수 있다.
3) 소화기계: 때때로 구역 등의 증상이 나타날 수 있다.
4) 피부: 드물게 접촉 피부염, 매우 드물게 혈관부종 등의 증상이 나타날 수 있다.
5) 호흡기계: 폐렴 등의 증상이 나타날 수 있다.
6) 내분비계: 매우
saved -> final_dosage_drug_ids_500.csv, final_context_packs.jsonl


- 청크별 사용횟수 확인

In [ ]:
def get_warn_index(chunk_id: str):
    """
    chunk_id에서 warn-숫자를 추출해서 int로 반환.
    예) 'xxxx-warn-2' -> 2
    """
    if not isinstance(chunk_id, str):
        return None
    m = re.search(r"-warn-(\d+)$", chunk_id)
    return int(m.group(1)) if m else None

df_warn = df_pack[df_pack["context_mode"] == "warning"].copy()

counter = Counter()

for ids in df_warn["source_chunk_ids"]:
    if not isinstance(ids, list):
        continue
    for cid in ids:
        idx = get_warn_index(cid)
        if idx is not None:
            counter[idx] += 1

freq = dict(sorted(counter.items(), key=lambda x: x[0]))
print("warn-청크 번호별 사용 횟수:")
for k, v in freq.items():
    print(f"warn-{k}: {v}")

warn-청크 번호별 사용 횟수:
warn-1: 1651
warn-2: 1395
warn-3: 1379
warn-4: 1160
warn-5: 1054
warn-6: 667
warn-7: 352
warn-8: 196
warn-9: 95
warn-10: 63
warn-11: 28
warn-12: 20
warn-13: 13
warn-14: 15
warn-15: 12
warn-16: 11
warn-17: 11
warn-18: 14
warn-19: 1
warn-20: 4
warn-21: 1
warn-25: 2
warn-32: 2
warn-37: 1
warn-42: 2


- warn 1-5까지만 존재하는 약물이 많아 1-5 문서가 가장 집중적으로 선택됨

## 3-2. 파인튜닝용 샘플 선택

In [ ]:
def make_ingredient_key(x):
    
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "UNK"

    s = x
    if isinstance(x, list):
        s = "|".join(map(str, x))
    s = str(s)

    # 첫 성분만 대표로 사용 (원하면 "정렬된 전체"로 바꿔도 됨)
    first = s.split("|")[0].strip()

    # 코드가 있으면 코드만, 없으면 텍스트 일부
    m = re.search(r"\[([A-Za-z0-9]+)\]", first)
    if m:
        return m.group(1)  # 예: M215884
    return first[:30] if first else "UNK"


def attach_warn_total_chunks(df_pack, df_chunk=None):
    df = df_pack.copy()

    if df_chunk is None:
        df["warn_total_chunks"] = 1
        return df

    tmp = df_chunk.copy()
    tmp = tmp[tmp["section"] == "주의사항"]

    tmp["sub_count"] = pd.to_numeric(tmp["sub_count"], errors="coerce")

    warn_tot = tmp.groupby("drug_id")["sub_count"].max().rename("warn_total_chunks")
    df = df.merge(warn_tot, on="drug_id", how="left")

    df["warn_total_chunks"] = df["warn_total_chunks"].fillna(1).astype(int)
    return df


def add_strata_columns(df, head_topk=120):
    out = df.copy()

    out["context_len"] = out["context"].astype(str).str.len()

    out["context_len_bin"] = pd.cut(
        out["context_len"],
        bins=[-1, 400, 800, 1200, 10**9],
        labels=["L0_400", "L401_800", "L801_1200", "L1201p"]
    ).astype(str)

    out["warn_bin"] = pd.cut(
        out["warn_total_chunks"],
        bins=[0, 1, 2, 4, 7, 10**9],
        labels=["W1", "W2", "W3_4", "W5_7", "W8p"],
        right=True
    ).astype(str)

    out["ingredient_key"] = out["active_ingredient"].apply(make_ingredient_key)
    freq = out["ingredient_key"].value_counts()
    head_set = set(freq.head(head_topk).index)
    out["ingredient_bucket"] = np.where(out["ingredient_key"].isin(head_set), "HEAD", "TAIL")

    return out


def stratified_sample(df, n, strata_cols, min_per_stratum=1, seed=42):
    """
    strata_cols 조합별로 최소 min_per_stratum 확보 후,
    남은 수량은 strata 크기 비율대로 배분해서 샘플링
    """
    rng = np.random.RandomState(seed)
    df = df.copy()

    df["_stratum"] = df[strata_cols].astype(str).agg("|".join, axis=1)

    groups = df.groupby("_stratum", sort=False)
    sizes = groups.size()

    num_strata = len(sizes)
    if num_strata * min_per_stratum > n:
        min_per_stratum = max(0, n // max(1, num_strata))

    picks = []
    used = set()
    if min_per_stratum > 0:
        for key, g in groups:
            k = min(min_per_stratum, len(g))
            if k > 0:
                sample_idx = g.sample(n=k, random_state=rng.randint(0, 1_000_000)).index
                picks.extend(sample_idx.tolist())
                used.update(sample_idx.tolist())

    remain = n - len(picks)
    if remain <= 0:
        return df.loc[picks].drop(columns=["_stratum"])

    df_rem = df.drop(index=list(used))
    groups_rem = df_rem.groupby("_stratum", sort=False)
    sizes_rem = groups_rem.size()

    if sizes_rem.sum() == 0:
        return df.loc[picks].drop(columns=["_stratum"])

    ratios = (sizes_rem / sizes_rem.sum())
    alloc = (ratios * remain).astype(int)

    short = remain - alloc.sum()
    if short > 0:
        for k in ratios.sort_values(ascending=False).index[:short]:
            alloc.loc[k] += 1

    for key, k in alloc.items():
        if k <= 0:
            continue
        g = groups_rem.get_group(key)
        k = min(k, len(g))
        if k > 0:
            sample_idx = g.sample(n=k, random_state=rng.randint(0, 1_000_000)).index
            picks.extend(sample_idx.tolist())

    return df.loc[picks].drop(columns=["_stratum"])


def build_tuning_sample(
    df_pack,
    df_chunk=None,
    total_n=1000,
    dosage_n=500,
    head_topk=120,
    min_per_stratum=2,
    seed=42
):

    df = attach_warn_total_chunks(df_pack, df_chunk=df_chunk)

    df = add_strata_columns(df, head_topk=head_topk)

    df_dos = df[df["context_mode"] == "dosage"].copy()
    df_war = df[df["context_mode"] == "warning"].copy()

    dosage_n = min(dosage_n, len(df_dos))
    warning_n = max(0, total_n - dosage_n)
    warning_n = min(warning_n, len(df_war))

    dos_sample = stratified_sample(
        df_dos,
        n=dosage_n,
        strata_cols=["context_len_bin", "ingredient_bucket"],
        min_per_stratum=max(1, min_per_stratum//2),
        seed=seed
    )

    war_sample = stratified_sample(
        df_war,
        n=warning_n,
        strata_cols=["warn_bin", "context_len_bin", "ingredient_bucket"],
        min_per_stratum=min_per_stratum,
        seed=seed + 1
    )

    out = pd.concat([dos_sample, war_sample], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)

    print("=== Tuning sample summary ===")
    print("total:", len(out), "| dosage:", (out["context_mode"]=="dosage").sum(), "| warning:", (out["context_mode"]=="warning").sum())
    print("\n[warning] warn_bin counts:\n", out[out["context_mode"]=="warning"]["warn_bin"].value_counts(dropna=False))
    print("\n[all] ingredient_bucket counts:\n", out["ingredient_bucket"].value_counts(dropna=False))
    print("\n[all] context_len_bin counts:\n", out["context_len_bin"].value_counts(dropna=False))

    return out


In [ ]:
df_tune = build_tuning_sample(
    df_pack=df_pack,
    df_chunk=df_chunk,
    total_n=1000,
    dosage_n=100,
    seed=42
)

=== Tuning sample summary ===
total: 1000 | dosage: 100 | warning: 900

[warning] warn_bin counts:
 warn_bin
W5_7    529
W8p     200
W3_4    111
W1       33
W2       27
Name: count, dtype: int64

[all] ingredient_bucket counts:
 ingredient_bucket
HEAD    703
TAIL    297
Name: count, dtype: int64

[all] context_len_bin counts:
 context_len_bin
L0_400       744
L401_800     151
L801_1200     66
L1201p        39
Name: count, dtype: int64


In [ ]:
df_tune.to_json("final_tuning_sample_1000.jsonl", orient="records", lines=True, force_ascii=False)